In [14]:
# IMPORTING REQUIRED MODULES

import os
import random
import numpy as np

from keras.utils import np_utils
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


In [15]:
# PREPARING DATA

data_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train'
pokemons = os.listdir(data_path)
print(pokemons)

image_data = []
labels = []
count = 0

for ix in pokemons:
    path = os.path.join(data_path, ix)
    print(path, count) 
    for im in os.listdir(path):
        try:
            img = image.load_img(os.path.join(path, im), target_size = (224,224))
            img_array = image.img_to_array(img)
            image_data.append(img_array)
            labels.append(count)
        except Exception as e:
            print(os.path.join(path, im))
            print(e)
    count += 1

combined_dataset = list(zip(image_data, labels))
random.shuffle(combined_dataset)
image_data[:], labels[:] = zip(*combined_dataset)

X_train = np.array(image_data)

Y_train = np.array(labels)
Y_train = np_utils.to_categorical(Y_train)


['Charmander', 'Fearow', 'Psyduck', 'Dratini', 'Squirtle', 'Meowth', 'Pikachu', 'Aerodactyl', 'Bulbasaur', 'Spearow']
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Charmander 0
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Fearow 1
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Fearow/8f2d7986ab76482fb76cac6a91b66de4.svg
cannot identify image file <_io.BytesIO object at 0x7f7f2c31b040>
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Fearow/2c8d5221e92444f2b9477350e405bc54.svg
cannot identify image file <_io.BytesIO object at 0x7f7f2c31b040>
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Psyduck 2
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Dratini 3
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Squirtle 4
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Meowth 5
/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Pikachu 6
/content/drive

In [16]:
# LOADING THE BASE MOBILENET FROM KERAS

mobile_net_model = MobileNet(include_top=False, weights='imagenet', input_shape = (224,224,3))
mobile_net_model.summary()


Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [17]:
# CREATING OUR OWN CLASSIFIER AND CONNECTING WITH MOBILENET

av1 = GlobalAveragePooling2D()(mobile_net_model.output)
fc1 = Dense(256, activation = 'relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(128, activation = 'relu')(d1)
d2 = Dropout(0.5)(fc2)
fc3 = Dense(64, activation = 'relu')(d2)
d3 = Dropout(0.5)(fc3)
fc4 = Dense(10, activation = 'softmax')(d3)

mobile_net_final_model = Model(inputs = mobile_net_model.input, outputs = fc4)
mobile_net_final_model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [18]:
# PREDICTING WITHOUT TRAINING OUR CLASSIFIER

image_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Pikachu/00000007.jpg'
img = image.load_img(image_path, target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)

pred = mobile_net_final_model.predict(x)
print(pokemons[np.argmax(pred)])

1/1 [==============================] - 0s 399ms/step
Meowth


In [19]:
# COMPILING MODEL FOR TRAINING

adam = Adam(lr = 0.00003)
mobile_net_final_model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [20]:
# CHECKING ALL LAYERS OF THE MODEL

for ix in range(len(mobile_net_final_model.layers)):
    print(ix, mobile_net_final_model.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f7f768b7c70>
1 <keras.layers.convolutional.conv2d.Conv2D object at 0x7f7ebdf25400>
2 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f7f767a2a00>
3 <keras.layers.activation.relu.ReLU object at 0x7f7f38038a90>
4 <keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x7f7ebdf1d760>
5 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f7ebdf2f2b0>
6 <keras.layers.activation.relu.ReLU object at 0x7f7ebdfbc340>
7 <keras.layers.convolutional.conv2d.Conv2D object at 0x7f7f75d25be0>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f7f75d294c0>
9 <keras.layers.activation.relu.ReLU object at 0x7f7f75d25af0>
10 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f7f75d29f10>
11 <keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x7f7f75d25580>
12 <keras.layers.normalization.batch_normalization.BatchNorm

In [21]:
# FREEZING ALL LAYERS EXCEPT A FEW FOR TRAINING ON DATASET

for ix in range(86):
    mobile_net_final_model.layers[ix].trainable = False
print(mobile_net_final_model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [22]:
# TRAINING ONLY TRAINABLE LAYERS OF MODEL

hist = mobile_net_final_model.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 30, validation_split = 0.20)

Epoch 1/30
83/83 [==============================] - 20s 81ms/step - loss: 2.4778 - accuracy: 0.2090 - val_loss: 1.8343 - val_accuracy: 0.3678
Epoch 2/30
83/83 [==============================] - 6s 69ms/step - loss: 1.6474 - accuracy: 0.4324 - val_loss: 1.7930 - val_accuracy: 0.4924
Epoch 3/30
83/83 [==============================] - 5s 66ms/step - loss: 1.3124 - accuracy: 0.5517 - val_loss: 2.2807 - val_accuracy: 0.5441
Epoch 4/30
83/83 [==============================] - 6s 66ms/step - loss: 1.2427 - accuracy: 0.6056 - val_loss: 1.0768 - val_accuracy: 0.6687
Epoch 5/30
83/83 [==============================] - 5s 66ms/step - loss: 1.0546 - accuracy: 0.6672 - val_loss: 1.0753 - val_accuracy: 0.6717
Epoch 6/30
83/83 [==============================] - 6s 69ms/step - loss: 1.0256 - accuracy: 0.6892 - val_loss: 1.1602 - val_accuracy: 0.7052
Epoch 7/30
83/83 [==============================] - 5s 65ms/step - loss: 0.8517 - accuracy: 0.7356 - val_loss: 1.0216 - val_accuracy: 0.7264
Epoch 8/30
8

In [24]:
# PREDICTING MODEL AFTER TRAINING LAST LAYERS WITH OUR DATA

# image_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/Train/Pikachu/00000007.jpg'
image_path = '/content/drive/MyDrive/Colab Notebooks/Pokemon Classifier/8f2d7986ab76482fb76cac6a91b66de4.png'
img = image.load_img(image_path, target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)

pred = mobile_net_final_model.predict(x)
print(pokemons[np.argmax(pred)])

1/1 [==============================] - 0s 21ms/step
Fearow
